In [1]:
# fine-tuning VGG16 和特征提取互补，微调是将顶部的前几层解冻，并将解冻的基层和新增的部分联合训练
# 基本步骤：
# 1.在已经训练好的基础网络上添加自定义网络
# 2.冻结基础网络
# 3.训练所添加的部分
# 4.解冻基础网络的一些层
# 5.联合训练解冻的这些曾和添加的部分

In [2]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.applications import VGG16
# 定义一个Dense层的分类器
from keras import layers
from keras import models
from keras import optimizers

Using TensorFlow backend.


In [3]:
train_dir = "./data/dogs-vs-cats/cats_and_dogs_small/train/"
validation_dir = "./data/dogs-vs-cats/cats_and_dogs_small/validation/"
test_dir = "./data/dogs-vs-cats/cats_and_dogs_small/test/"

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=40,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20
)

Found 2000 images belonging to 2 classes.


In [6]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20
)

Found 1000 images belonging to 2 classes.


In [7]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=20
)

Found 1000 images belonging to 2 classes.


In [8]:
model = models.Sequential()

In [9]:
model.add(conv_base)

In [10]:
model.add(layers.Flatten())

In [11]:
model.add(layers.Dense(256, activation='relu'))

In [12]:
model.add(layers.Dense(1, activation='sigmoid'))

In [13]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [14]:
# 微调 block5_conv1,block5_conv2, block5_conv3三层;其余冻结
# 卷积基中更靠近底部的编码是更加通用和可复用的特征，而靠近顶部的层编码是更专业化的特征。
# 微调这些更专业的特征更有用，因为他们需要在你的新问题上改变用途，微调更靠近底部的层得到的回报会越少
# 只微调最后的三层是为了防止过拟合，因为VGG16 在 猫狗数据集上有1500W个参数

In [15]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == "block5_conv1":
        set_trainable = True
    if set_trainable is True:
        layer.trainable = True
    else:
        layer.trainable = False

In [16]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [17]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'], loss='binary_crossentropy')

In [20]:
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=30, validation_data=validation_generator, validation_steps=50)

Epoch 1/30
100/100 [==============================] - 18s 183ms/step - loss: 0.4908 - acc: 0.7445 - val_loss: 0.2678 - val_acc: 0.8850
Epoch 2/30
100/100 [==============================] - 16s 164ms/step - loss: 0.3249 - acc: 0.8525 - val_loss: 0.2141 - val_acc: 0.9060
Epoch 3/30
100/100 [==============================] - 16s 164ms/step - loss: 0.2869 - acc: 0.8725 - val_loss: 0.1871 - val_acc: 0.9240
Epoch 4/30
100/100 [==============================] - 16s 165ms/step - loss: 0.2470 - acc: 0.9005 - val_loss: 0.1845 - val_acc: 0.9280
Epoch 5/30
100/100 [==============================] - 16s 165ms/step - loss: 0.2186 - acc: 0.9140 - val_loss: 0.2267 - val_acc: 0.9100
Epoch 6/30
100/100 [==============================] - 17s 174ms/step - loss: 0.1966 - acc: 0.9155 - val_loss: 0.1621 - val_acc: 0.9340
Epoch 7/30
100/100 [==============================] - 16s 165ms/step - loss: 0.1952 - acc: 0.9185 - val_loss: 0.1851 - val_acc: 0.9250
Epoch 8/30
100/100 [==============================] - 1

In [21]:
model.evaluate_generator(test_generator)

[0.27879618115810445, 0.9419999933242797]